In [ ]:
from PIL import Image
import cv2
import numpy as np
import os
from models import UNet
from helpers import pad_to_square_multiple

In [ ]:
model = UNet.load("UNet_3")

# Extract map features

In [ ]:
layout_path = "data/layout"
for file in os.listdir(f"{layout_path}/raw")[0:1]:
    path = os.path.join(f"{layout_path}/raw", file)
    x = Image.open(path)
    x = np.array(x)[:,:,:3]
    chunk_size = 64
    x = pad_to_square_multiple(x, chunk_size)
    x = model.batch_inference(x, chunk_size=chunk_size)
    output_path = os.path.join(f"{layout_path}/extract", os.path.splitext(file)[0] + ".npz")
    np.savez_compressed(output_path, data=x)

In [ ]:
ex = 2
path = f"data/layout/raw/coast-{ex}.png"
display(Image.open(path))
output_path = f"data/layout/extract/coast-{ex}.npz"
display(Image.fromarray(np.load(output_path)['data'] * 255, mode="L"))

# Manually label layout classes

In [ ]:
def resize_square(image, max_size):
    image = image.astype(np.uint8)
    height, width = image.shape
    scale = min(max_size / width, max_size / height)
    new_width = int(width * scale)
    new_height = int(height * scale)
    block_height = height // new_height
    block_width = width // new_width
    reshaped = image[:block_height*new_height, :block_width*new_width]
    reshaped = reshaped.reshape(new_height, block_height, new_width, block_width)
    pooled = reshaped.max(axis=(1, 3))
    square_image = np.zeros((max_size, max_size), dtype=np.uint8)
    pad_y = (max_size - new_height) // 2
    pad_x = (max_size - new_width) // 2
    square_image[pad_y:pad_y+new_height, pad_x:pad_x+new_width] = pooled
    return square_image

In [ ]:
truth_labels = [
    0, 1, 0, 2,
    3, 2, 4, 5,
    6, 6, 7, 3,
    5, 7, 8,
]
layouts = []
layout_dir = "data/layout/extract"
for file in sorted(os.listdir(layout_dir), key=lambda x: int(x.split('coast-')[1].split('.npz')[0])):
    layout = np.load(os.path.join(layout_dir, file))['data']
    layouts.append(resize_square(layout, 400))

row_size = 4
for i in range(0, len(layouts), row_size):
    row = np.concatenate(layouts[i:i + row_size], axis=1)
    display(Image.fromarray(row * 255, mode="L"))
    labels = truth_labels[i:i+row_size]
    print(labels)

In [ ]:
truth_labels = [
    0, 1, 0, 2,
    3, 2, 4, 5,
    6, 6, 7, 3,
    5, 7, 8,
]
layouts = []
layout_dir = "data/layout/extract"
for file in sorted(os.listdir(layout_dir), key=lambda x: int(x.split('coast-')[1].split('.npz')[0])):
    layout = np.load(os.path.join(layout_dir, file))['data']
    layouts.append(resize_square(layout, 400))

grouped = {}
for i, label in enumerate(truth_labels):
    if grouped.get(label) is None:
        grouped[label] = []
    grouped[label].append(layouts[i])

In [ ]:
unique_layouts, unique_labels = [], []
for group in grouped:
    unique_layouts.append(grouped[group][0])
    unique_labels.append(group)
                          
row_size = 4
for i in range(0, len(unique_layouts), row_size):
    row = np.concatenate(unique_layouts[i:i + row_size], axis=1)
    display(Image.fromarray(row * 255, mode="L"))
    labels = unique_labels[i:i+row_size]
    print(labels)